In [161]:
import pandas as pd
import random
from copy import deepcopy
import numpy as np

In [162]:
X_train = pd.read_csv('data/X_train.csv')
X_test  = pd.read_csv('data/X_test.csv')

y_train = pd.read_csv('data/y_train.csv')
y_test  = pd.read_csv('data/y_test.csv')

In [163]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [164]:
class Individual:
    def __init__(self,code,fitness = -float("inf")):
        self.code = code
        self.fitness = fitness

In [165]:
class GA:

    def __init__(self, X:pd.DataFrame, y:pd.DataFrame, population_size:int, num_generations:int, tournament_size:int, mutation_probability:float):
        #dodaj elitizam
        self.X = X
        self.y = y
        self.all_column_names = X.columns
        self.population_size = population_size
        self.num_generations = num_generations
        self.tournament_size = tournament_size
        self.mutation_probability = mutation_probability
        
        
        self.population = [ Individual(code=(np.random.rand(X.shape[1]) < 0.5)) for _ in range(self.population_size) ]
        self.best_fitness = -float('inf')
        self.best = None
        self.history = []

    def calc_fitness(self,index): 
        if not any(self.population[index].code):
            self.population[index].fitness =  -float('inf')
            return
        
        new_column_names = [j for i,j in enumerate(self.all_column_names) if self.population[index].code[i]]
    
        model = SVC(kernel='linear',probability=False)
        self.y = self.y.squeeze()
        model.fit(self.X[new_column_names],self.y)
        y_pred = model.predict(self.X[new_column_names]) #train test split
        acc = accuracy_score(y_pred,self.y)
        
        num_features = self.population[index].fitness =  sum(self.population[index].code)
        self.population[index].fitness = acc + (1 - num_features / self.X.shape[1])   
        # a*tacnost + b * (1 - izabrani_atributi / ukupno_atributa )
        #ili a * tacnost + (1 - a) ...
        #mozda da oduzimam drugi deo
        

    def crossover(self,parent1,parent2):
        breakpoint = random.randrange(1, len(parent1.code))
        child1 = Individual(np.concatenate([parent1.code[:breakpoint],parent2.code[breakpoint:]]) )
        child2 =  Individual(np.concatenate([parent2.code[:breakpoint],parent1.code[breakpoint:]]) )

        return child1,child2

    def mutation(self,child):
        for i in range(len(child.code)):
            if random.random() < self.mutation_probability:
                child.code[i] = not child.code[i]
        return child
        
    def tournament_selection(self):
        contenders = random.sample(range(len(self.population)), self.tournament_size)
        return max(contenders, key=lambda i: self.population[i].fitness)
        

    def run(self):
               
        for generation in range(self.num_generations):
            
            for i in range(self.population_size):
                self.calc_fitness(i)
                if self.population[i].fitness > self.best_fitness:
                    self.best_fitness = self.population[i].fitness
                    self.best = self.population[i].code.copy()
            
            self.history.append((generation,self.best_fitness,self.best))
            new_population = []
            for i in range(0,self.population_size,2):
                parent1 = self.population[self.tournament_selection()]
                parent2 = self.population[self.tournament_selection()]
                child1, child2 = self.crossover(parent1, parent2)
                new_population.append(self.mutation(child1))
                if len(new_population) < self.population_size: #unapredi ovo
                    new_population.append(self.mutation(child2))
            self.population = deepcopy(new_population)

    

        return self.best, self.best_fitness #, history
    

In [166]:
ga = GA(X_train,y_train,population_size = 100, num_generations = 10, tournament_size=4, mutation_probability = 0.05)
ga.run()

(array([ True, False,  True, ..., False,  True, False]), 1.536400617197363)

In [160]:
ga.history

[(0,
  1.5164819750315612,
  array([False,  True, False, ...,  True, False, False])),
 (1,
  1.5191471454621968,
  array([ True, False, False, ...,  True, False,  True])),
 (2,
  1.5201290503576939,
  array([ True, False,  True, ...,  True, False, False])),
 (3,
  1.5247580305793238,
  array([ True, False,  True, ...,  True,  True,  True])),
 (4,
  1.5281245616496002,
  array([ True, False,  True, ...,  True, False,  True])),
 (5,
  1.5281245616496002,
  array([ True, False,  True, ...,  True, False,  True])),
 (6,
  1.5327535418712301,
  array([ True, False,  True, ...,  True, False,  True])),
 (7,
  1.533454902510871,
  array([False, False, False, ..., False,  True,  True])),
 (8,
  1.536400617197363,
  array([ True, False,  True, ..., False,  True,  True])),
 (9,
  1.536400617197363,
  array([ True, False,  True, ..., False,  True,  True])),
 (10,
  1.5376630663487165,
  array([False, False, False, ...,  True,  True,  True])),
 (11,
  1.5378033384766447,
  array([False, False, False